In [47]:
import unittest
import requests
import re
import csv
import time
from unittest.mock import patch, MagicMock

In [48]:
# 爬虫-单元测试

In [49]:
# 被测试的爬虫代码，这里为了简化，只测试所爬取的第一页
def crawl_github():
    base_url = "https://github.com/crystal-lang/crystal/issues?page={}"

    headers = {
        'Cookie': '_device_id=4fa68362950d4d06252c186eaad61d85; _octo=GH1.1.549837167.1736940028; saved_user_sessions=187375867%3AIUsH8ag-p_jicReXL3fgh2NEynKEClXAta4lRW9NOIYjp8_J; user_session=IUsH8ag-p_jicReXL3fgh2NEynKEClXAta4lRW9NOIYjp8_J; __Host-user_session_same_site=IUsH8ag-p_jicReXL3fgh2NEynKEClXAta4lRW9NOIYjp8_J; logged_in=yes; dotcom_user=xiaoyucodequeen; color_mode=%7B%22color_mode%22%3A%22auto%22%2C%22light_theme%22%3A%7B%22name%22%3A%22light%22%2C%22color_mode%22%3A%22light%22%7D%2C%22dark_theme%22%3A%7B%22name%22%3A%22dark%22%2C%22color_mode%22%3A%22dark%22%7D%7D; cpu_bucket=xlg; preferred_color_mode=dark; tz=Asia%2FShanghai; _gh_sess=R4oZPumE2ozdFyeDlinh%2FkUeMKGqsxKWvJmyZRQr4L9cJaxN9XHOehx7JUqF5e52sbnIkvwdxxquSH7MMySuZEc%2FWZtogBR85kaq8WVjgR4Aygs93pAI9xdKVs8LtJAKi0JDntxdztGfcuR611oaBjoYl33dh4FvKXza9NxmGw8llOyyQGSisL1vDM4Cz8njsN%2F6QXoE7VXwh%2BXDy7aV7zrFEmyOwd1Qvr8BeiTwcqHCbh549KpwIMywhWNSRO0i%2BxNutZI1zuBsusvv388Fa8iS%2BaEZ3J0QsH4YmWDOgsA9nuteu04ik6rDcQVEK4VxAWYzB6bjVl%2BFg8BcqMEoSLVPDA6UJpGB2speudf7w4sVy5zflzvk3%2BG70zo2%2BnWH%2FR6qJqn%2Bs5TEiv48ypW%2FKbKSUKg%3D--QyIaHK4UzwjzpghW--%2BFUuMXV7jiZiVHQbkUV%2F0w%3D%3D',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0'
    }
    
    # 存储数据
    data = []
    
    # 循环爬取每一页
    for page in range(1, 2):  
        #print(f"正在爬取第 {page} 页...")
        url = base_url.format(page)
    
        # 发送 GET 请求
        response = requests.get(url=url, headers=headers, verify=False)
    
        if response.status_code != 200:
            print(f"请求失败，状态码: {response.status_code}")
            continue
    
        content = response.content.decode('utf8')
    
        update_content_temp = re.findall('<span class="Text__StyledText-sc-17v1xeu-0 hWqAbU">(.*?)</span>', content, re.DOTALL)
        update_type = re.findall(
            '<div class="Box-sc-g0xbh4-0 TrailingBadge-module__container--gg6pc" data-listview-component="trailing-badge">.*?<span class="Text__StyledText-sc-17v1xeu-0 hWqAbU">(.*?)</span>',
            content, re.DOTALL)
        update_content = list(set(update_content_temp)-set(update_type))
        view_count = re.findall('<span class="issue-item-module__defaultNumberDescription--GXzri">.*?<!.*?>(.*?)</span>',
                                content, re.DOTALL)
        update_time = re.findall('<relative-time class="sc-aXZVg".*?<!.*?>(.*?)</relative-time>', content, re.DOTALL)
    
        for i in range(len(update_content)):
            data.append({
                'Update Content': update_content[i],
                'Update Type': update_type[i] if i < len(update_type) else None,
                'View Count': view_count[i] if i < len(view_count) else None,
                'Update Time': update_time[i] if i < len(update_time) else None,
            })
    
        time.sleep(1)  # 延迟1秒以避免过于频繁的请求
    return data


In [50]:
data = crawl_github()
data

D:\python\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'Update Content': 'splatting into variadic C function yields a BUG',
  'Update Type': 'good first issue',
  'View Count': '15356',
  'Update Time': 'Jan 19, 2025'},
 {'Update Content': 'Codegen bug with duplicate ivar declaration and inheriting from generic instance type',
  'Update Type': 'kind:bug',
  'View Count': '15353',
  'Update Time': 'Jan 17, 2025'},
 {'Update Content': 'Expose method to process HTTP requests on HTTP::Server',
  'Update Type': 'topic:tools:docs-generator',
  'View Count': '15352',
  'Update Time': 'Jan 17, 2025'},
 {'Update Content': 'Unhelpful error message (<code>Error: unexpected token: "("</code>) when defining method that starts with a capital letter.',
  'Update Type': 'kind:bug',
  'View Count': '15342',
  'Update Time': 'Jan 14, 2025'},
 {'Update Content': '<code>Enumerable#sum(&amp;)</code> fails with certain block arguments',
  'Update Type': 'topic:compiler:cli',
  'View Count': '15338',
  'Update Time': 'Jan 12, 2025'},
 {'Update Content': 'Bette

In [51]:
length = len(data)
length

12

In [55]:
# 异常处理--请求失败处理
# 检查爬取数据过程中出现异常时，是否给予正确的错误提示
class TestGithubCrawlerException(unittest.TestCase):
    @patch('requests.get')
    def test_request_failure(self, mock_get):
        mock_get.return_value.status_code = 404
        # 调用 crawl_github 并存储打印输出
        import io
        from contextlib import redirect_stdout
        f = io.StringIO()
        with redirect_stdout(f):
            crawl_github()
        output = f.getvalue()
        # 检查是否打印了 "请求失败"
        self.assertIn("请求失败", output)

In [56]:
# 测试类--单元测试
class TestGithubCrawler(unittest.TestCase):
    # 模拟requests.get函数，避免真正的网络请求
    @patch('requests.get')
    def test_crawl_github(self, mock_get):
        # 模拟响应
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.content = b'<span class="Text__StyledText-sc-17v1xeu-0 hWqAbU">Test Content</span><div class="Box-sc-g0xbh4-0 TrailingBadge-module__container--gg6pc" data-listview-component="trailing-badge"><span class="Text__StyledText-sc-17v1xeu-0 hWqAbU">Test Type</span></div><span class="issue-item-module__defaultNumberDescription--GXzri">100</span><relative-time class="sc-aXZVg">2025-01-20</relative-time>'
        mock_get.return_value = mock_response

        result = data
        self.assertEqual(len(data), 12)
        # 对爬取结果的一到三行进行测试
        self.assertEqual(data[0]['Update Content'], 'splatting into variadic C function yields a BUG')
        self.assertEqual(data[0]['Update Type'], 'good first issue')
        self.assertEqual(data[0]['View Count'], '15356')
        self.assertEqual(data[0]['Update Time'], 'Jan 19, 2025')

        self.assertEqual(data[1]['Update Content'], 'Codegen bug with duplicate ivar declaration and inheriting from generic instance type')
        self.assertEqual(data[1]['Update Type'], 'kind:bug')
        self.assertEqual(data[1]['View Count'], '15353')
        self.assertEqual(data[1]['Update Time'], 'Jan 17, 2025')

        self.assertEqual(data[2]['Update Content'], 'Expose method to process HTTP requests on HTTP::Server')
        self.assertEqual(data[2]['Update Type'], 'topic:tools:docs-generator')
        self.assertEqual(data[2]['View Count'], '15352')
        self.assertEqual(data[2]['Update Time'], 'Jan 17, 2025')

In [57]:
# 运行单元测试
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK
